In [5]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [6]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [13]:
aggr = [{"$match" : {"soort" : "Vondst"}},
        {"$group" : {"_id" : {"projectcd" : "$projectcd","vondstnr" : "$vondstnr"}, "putnrs" : {"$addToSet" : "$putnr"}}},
        {"$replaceRoot" : {"newRoot" : {"_id" : 0,"projectcd" : "$_id.projectcd","aantal_put" : {"$size" : "$putnrs"}}}},
        {"$match" : {"aantal_put" : {"$gt" : 1.0}}},
        {"$group" : {"_id" : "$projectcd"}},
        {"$replaceRoot" : {"newRoot" : {"projectcd" : "$_id"}}}]


list(pd.DataFrame(list(analyseCol.aggregate(aggr)))['projectcd'])

['DB34']

In [16]:
aggr_vondstnr_not_unique = [{"$match" : {"soort" : "Vondst"}},
        {"$group" : {"_id" : {"projectcd" : "$projectcd","vondstnr" : "$vondstnr"}, "putnrs" : {"$addToSet" : "$putnr"}}},
        {"$replaceRoot" : {"newRoot" : {"_id" : 0,"projectcd" : "$_id.projectcd","aantal_put" : {"$size" : "$putnrs"}}}},
        {"$match" : {"aantal_put" : {"$gt" : 1.0}}},
        {"$group" : {"_id" : "$projectcd"}},
        {"$replaceRoot" : {"newRoot" : {"projectcd" : "$_id"}}}]

def setAndVondstUniqueInProject(col='analyse'):
    try:        
        if (col == 'analyse'):
            collection = analyseCol
        elif (col == 'analyseclean'):
            collection = analyseColClean
        else:
            raise ValueError('Error: Herkent de collectie niet met naam ' + col)

        lst_project = list(col.find({'soort': 'Artefact'}).distinct('projectcd'))
        for proj in lst_project:
            try:
                df_art = pd.DataFrame(list(col.find({'soort': 'Artefact', 'projectcd': proj}, projection={'artefactnr':1}))).dropna()
                unique = df_art['artefactnr'].is_unique
                
                project = col.find_one({ 'soort': "project", 'projectcd': proj })
                project['artefactnrs_unique'] = unique
                col.replace_one({'_id': project['_id']}, project)

            except Exception as exp2:
                logger.error(f'Error while determining whether artefactnr are unique for project {proj} with message: {str(exp2)} ')


        lst_project = list(col.find({'soort': 'Project'}))
        lst_vondst_in_multi_putten = list(pd.DataFrame(list(analyseCol.aggregate(aggr)))['projectcd'])
        for project in lst_project:
            try:
                project['vondstnr_unique'] = False if project['projectcd'] in lst_vondst_in_multi_putten else True
                col.replace_one({'_id': project['_id']}, project)

            except Exception as exp2:
                logger.error(f'Error while determining whether artefactnr and vondstnr are unique for project {proj} with message: {str(exp2)} ')
    except Exception as exp1:
        logger.error(f'Severe rrror while determining whether artefactnr and vondstnr are unique with message: {str(exp1)} ')
